Importing libraries

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from constants import *
import torch.optim as optim

import matplotlib.pyplot as plt
import numpy as np
import sklearn.metrics as metrics
import os
import psutil

import torch
import torchvision.transforms as transforms

from PIL import Image

import matplotlib.image as mpimg
import re

#!/usr/bin/python
import os
import sys
from PIL import Image
import math
import matplotlib.image as mpimg
import numpy as np

from torchsummary import summary

Constants

In [2]:
BATCH_SIZE = 1
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#Whether to display the logs during training
DISPLAY = False
TRAINING_SIZE = 4 # Debug purposes
NUM_EPOCHS = 3
N_CLASSES = 2
NR_TEST_IMAGES = 2 # 50
rotateFlag = False
RATIO=0.5
IMG_PATCH_SIZE = 16

submissionFileName = "latestSubmission.csv"
data_dir = '../Datasets/training/'
test_dir = '../Datasets/test_set_images/test_'
train_data_filename = 'images/'
train_labels_filename = 'groundtruth/'
RELOAD_MODEL = False
MODEL_PATH = "./model.pkg"

Creating the models

In [3]:
# -------------------------------------------
# U-NET
# Architecture based on the initial paper
# -------------------------------------------
class UNET(nn.Module):
    def __init__(self, in_channels = 3, out_channels = 2):
        super(UNET,self).__init__()
        # Contracting path
        self.contract1 = self.doubleConv_block(in_channels, 64)
        self.contract2 = self.doubleConv_block(64, 128)
        self.contract3 = self.doubleConv_block(128, 256)
        self.contract4 = self.doubleConv_block(256, 512)

        self.maxpool = torch.nn.MaxPool2d(kernel_size = 2)

        # Expansive path
        self.expand5 = self.expanding_block(512, 1024)
        self.expand4 = self.expanding_block(1024, 512)
        self.expand3 = self.expanding_block(512, 256)
        self.expand2 = self.expanding_block(256, 128)

        # Final block
        self.output = self.output_block(128, out_channels)


    def doubleConv_block(self, in_channels, out_channels):
        """ (conv + ReLU + BN) * 2 times """
        doubleConv_block = torch.nn.Sequential(torch.nn.Conv2d(in_channels, out_channels, kernel_size = 3),
        torch.nn.ReLU(),  # apply activation function
        # Position of Batch Normalization (BN) wrt nonlinearity unclear, but experiments are generally in favor of this solution, which is the current default(ReLU + BN)
        torch.nn.BatchNorm2d(out_channels),
        torch.nn.Conv2d(out_channels, out_channels, kernel_size = 3),
        torch.nn.ReLU(),
        torch.nn.BatchNorm2d(out_channels))

        return doubleConv_block



    def expanding_block(self, in_channels, tmp_channels):
        """ (conv + ReLU + BN) * 2 times + upconv """
        out_channels = tmp_channels // 2
        expanding_block = torch.nn.Sequential(
            self.doubleConv_block(in_channels, tmp_channels),
            torch.nn.ConvTranspose2d(tmp_channels, out_channels, kernel_size = 2, stride = 2)
        )
        return expanding_block


    def output_block(self, in_channels = 128, out_channels = 2):
        tmp_channels = in_channels // 2
        output_block = torch.nn.Sequential(
            self.doubleConv_block(in_channels, tmp_channels),
            torch.nn.Conv2d(in_channels = tmp_channels, out_channels = out_channels, kernel_size = 1),
        )
        return output_block

    def concatenating_block(self, x_contracting, x_expanding):
        delta2 = (x_contracting.size()[2] - x_expanding.size()[2])
        delta = delta2 // 2
        if delta2 % 2 == 0 :
        # see which type of padding to apply
            x_cropped = F.pad(x_contracting, (-delta, -delta, -delta, -delta))
        else :
            x_cropped = F.pad(x_contracting, (-delta - 1, -delta, -delta - 1, -delta))
        return torch.cat([x_cropped, x_expanding], dim = 1)


    def forward(self, layer0):
        # Padding with reflection
        # pad size found such that after doing all the convolutions n_out = n_in <=> n_padded = n_in + 194 (to be verified)
        # Can't obtain a perfect padding to obtain a 200 * 200 image in the end : pad of 93 => final image 388 * 388 / pad of 94 => 404 * 404
        pad = 94
        layer0 = F.pad(layer0, (pad, pad, pad, pad), mode = 'reflect')
        print('layer0', layer0.shape)

        layer1_descending = self.contract1(layer0)
        print('layer1d', layer1_descending.shape)
        layer2_descending = self.contract2(self.maxpool(layer1_descending))
        print('layer2d', layer2_descending.shape)
        layer3_descending = self.contract3(self.maxpool(layer2_descending))
        print('layer3d', layer3_descending.shape)
        layer4_descending = self.contract4(self.maxpool(layer3_descending))
        print('layer4d', layer4_descending.shape)
        layer5 = self.maxpool(layer4_descending)
        print('layer5', layer5.shape)


        # _ascending = input of the layer
        layer4_ascending = self.expand5(layer5)
        print('layer4a', layer4_ascending.shape)
        layer3_ascending = self.expand4(self.concatenating_block(layer4_descending, layer4_ascending))
        print('layer3a', layer3_ascending.shape)
        layer2_ascending = self.expand3(self.concatenating_block(layer3_descending, layer3_ascending))
        print('layer2a', layer2_ascending.shape)
        layer1_ascending = self.expand2(self.concatenating_block(layer2_descending, layer2_ascending))
        print('layer1a', layer1_ascending.shape)

        output = self.output(self.concatenating_block(layer1_descending, layer1_ascending))
        print('output', output.shape)

        # Converting into vector
        #output = output.view(TRAINING_SIZE, N_CLASSES, -1)
        #print('output vector', output.shape)
        #finally no need to convert into vector with nn.crossentropy.loss

        return output


def create_UNET():
    network = UNET()
    network.to(DEVICE)
    return network


class smaller_UNET(nn.Module):
    def __init__(self, in_channels = 3, out_channels = 2):
        super(smaller_UNET,self).__init__()
        # Contracting path
        self.contract1 = self.doubleConv_block(in_channels, 32)
        self.contract2 = self.doubleConv_block(32, 64)
        self.contract3 = self.doubleConv_block(64, 128)
        self.contract4 = self.doubleConv_block(128, 256)

        self.maxpool = torch.nn.MaxPool2d(kernel_size = 2)

        # Expansive path
        self.expand5 = self.expanding_block(256, 512)
        self.expand4 = self.expanding_block(512, 256)
        self.expand3 = self.expanding_block(256, 128)
        self.expand2 = self.expanding_block(128, 64)

        # Final block
        self.output = self.output_block(64, out_channels)


    def doubleConv_block(self, in_channels, out_channels):
        """ (conv + ReLU + BN) * 2 times """
        doubleConv_block = torch.nn.Sequential(torch.nn.Conv2d(in_channels, out_channels, kernel_size = 3),
        torch.nn.ReLU(),  # apply activqtion function
        # Position of Batch Normalization (BN) wrt nonlinearity unclear, but experiments are generally in favor of this solution, which is the current default(ReLU + BN)
        torch.nn.BatchNorm2d(out_channels),
        torch.nn.Conv2d(out_channels, out_channels, kernel_size = 3),
        torch.nn.ReLU(),
        torch.nn.BatchNorm2d(out_channels))

        return doubleConv_block



    def expanding_block(self, in_channels, tmp_channels):
        """ (conv + ReLU + BN) * 2 times + upconv """
        out_channels = tmp_channels // 2
        expanding_block = torch.nn.Sequential(
            self.doubleConv_block(in_channels, tmp_channels),
            torch.nn.ConvTranspose2d(tmp_channels, out_channels, kernel_size = 2, stride = 2)
        )
        return expanding_block


    def output_block(self, in_channels = 128, out_channels = 2):
        tmp_channels = in_channels // 2
        output_block = torch.nn.Sequential(
            self.doubleConv_block(in_channels, tmp_channels),
            torch.nn.Conv2d(in_channels = tmp_channels, out_channels = out_channels, kernel_size = 1),
        )
        return output_block

    def concatenating_block(self, x_contracting, x_expanding):
        delta2 = (x_contracting.size()[2] - x_expanding.size()[2])
        delta = delta2 // 2
        if delta2 % 2 == 0 :
        # see which type of padding to apply
            x_cropped = F.pad(x_contracting, (-delta, -delta, -delta, -delta))
        else :
            x_cropped = F.pad(x_contracting, (-delta - 1, -delta, -delta - 1, -delta))
        return torch.cat([x_cropped, x_expanding], dim = 1)


    def forward(self, layer0):
        # Padding with reflection
        # pad size found such that after doing all the convolutions n_out = n_in <=> n_padded = n_in + 194 (to be verified)
        # Can't obtain a perfect padding to obtain a 200 * 200 image in the end : pad of 93 => final image 388 * 388 / pad of 94 => 404 * 404
        pad = 94
        layer0 = F.pad(layer0, (pad, pad, pad, pad), mode = 'reflect')
        print('layer0', layer0.shape)

        layer1_descending = self.contract1(layer0)
        print('layer1d', layer1_descending.shape)
        layer2_descending = self.contract2(self.maxpool(layer1_descending))
        print('layer2d', layer2_descending.shape)
        layer3_descending = self.contract3(self.maxpool(layer2_descending))
        print('layer3d', layer3_descending.shape)
        layer4_descending = self.contract4(self.maxpool(layer3_descending))
        print('layer4d', layer4_descending.shape)
        layer5 = self.maxpool(layer4_descending)
        print('layer5', layer5.shape)


        # _ascending = input of the layer
        layer4_ascending = self.expand5(layer5)
        print('layer4a', layer4_ascending.shape)
        layer3_ascending = self.expand4(self.concatenating_block(layer4_descending, layer4_ascending))
        print('layer3a', layer3_ascending.shape)
        layer2_ascending = self.expand3(self.concatenating_block(layer3_descending, layer3_ascending))
        print('layer2a', layer2_ascending.shape)
        layer1_ascending = self.expand2(self.concatenating_block(layer2_descending, layer2_ascending))
        print('layer1a', layer1_ascending.shape)

        output = self.output(self.concatenating_block(layer1_descending, layer1_ascending))
        print('output', output.shape)

        # Converting into vector
        #output = output.view(TRAINING_SIZE, N_CLASSES, -1)
        #print('output vector', output.shape)
        #finally no need to convert into vector with nn.crossentropy.loss

        return output


def create_smallerUNET():
    network = smaller_UNET()
    network.to(DEVICE)
    return network


Training

Scoring functions

In [4]:
# Chosen score : F1 metrics to be in accordance with AIcrowd

def score(y_true,y_pred_onehot):

    softMax = torch.nn.Softmax(1)
    y_pred = torch.argmax(softMax(y_pred_onehot),1)

    y_true_0 = y_true == False
    y_pred_0 = y_pred == False

    y_true_1 = y_true == True
    y_pred_1 = y_pred == True

    true_negative_nb = len(y_true[y_true_0 & y_pred_0])
    false_negative_nb = len(y_true[y_true_1 & y_pred_0])
    true_positive_nb = len(y_true[y_true_1 & y_pred_1])
    false_positive_nb = len(y_true[y_true_0 & y_pred_1])
    
    eps = 1 # to avoid division by zero 
    # to be changed !

    precision = true_positive_nb /(true_positive_nb + false_positive_nb + eps)
    recall = true_positive_nb / (true_positive_nb + false_negative_nb + eps)

    f1 = 2 * (precision * recall) / (precision + recall)
    return f1

def split_data(x,y,ratio, seed = 1):
    print(x.shape)
    """split the dataset based on the split ratio."""
    # set seed
    np.random.seed(seed)
    # generate random indices
    num_img = x.shape[0]
    indices = np.random.permutation(num_img)
    index_split = int(np.floor(ratio * num_img))
    index_tr = indices[: index_split]
    index_val = indices[index_split:]
    # create split
    x_tr = x[index_tr]
    x_val = x[index_val]
    y_tr = y[index_tr]
    y_val = y[index_val]
    return x_tr, x_val, y_tr, y_val

Actual training

In [5]:
def training(model, loss_function, optimizer, x, y, epochs, ratio):
    val_loss_hist = []
    val_acc_hist = []
    train_acc_hist = []
    train_loss_hist = []

    x,val_x,y, val_y = split_data(x, y, ratio)
    process = psutil.Process(os.getpid())

    for epoch in range(epochs):
        model.train()
        
        print("Training, epoch=", epoch)
        print("Memory usage {0:.2f} GB".format(process.memory_info().rss/1024/1024/1024))
        loss_value = 0.0
        correct = 0
        for i in range(0,x.shape[0],BATCH_SIZE):
            data_inputs = x[i:BATCH_SIZE+i].to(DEVICE)
            data_targets = y[i:BATCH_SIZE+i].to(DEVICE)

            print("Training image ", str(i))
            print("Memory usage {0:.2f} GB".format(process.memory_info().rss/1024/1024/1024))

            #Traning step
            optimizer.zero_grad()
            outputs = model(data_inputs)
            loss = loss_function(outputs, data_targets)
            loss.backward()
            optimizer.step()

            #Log
            loss_value += loss.item()
            correct += score(data_targets,outputs)

        loss_value /= x.shape[0]
        accuracy = correct/x.shape[0]

        #Validation prediction
        
        model.eval()
        loss_val_value = 0.0
        correct_val = 0
        for i in range(0,val_x.shape[0],BATCH_SIZE):
           
            
            data_val_inputs = val_x[i:BATCH_SIZE+i].to(DEVICE)
            data_val_targets = val_y[i:BATCH_SIZE+i].to(DEVICE)
        
            outputs_val = model(data_val_inputs)
            val_loss = loss_function(outputs_val,data_val_targets)
            
             # log 
            loss_val_value +=val_loss.item()
            correct_val += score(data_val_targets,outputs_val)
        
        
        loss_val_value /= val_x.shape[0]
        accuracy_val = correct_val/val_x.shape[0]

        #Log
        val_loss_hist.append(loss_val_value)
        val_acc_hist.append(accuracy_val)
        train_loss_hist.append(loss_value)
        train_acc_hist.append(accuracy)

        if DISPLAY:
            print(f'Epoch {epoch}, loss: {loss_value:.5f}, accuracy: {accuracy:.3f}, Val_loss: {loss_val_value:.5f}, Val_acc: {accuracy_val:.3f}')

        print(">> Saving Model for Epoch ", str(epoch))
        checkpoint = {'model_state': model.state_dict(), 'optimizer_state': optimizer.state_dict()}
        torch.save(checkpoint, './model')

    return val_loss_hist,train_loss_hist,val_acc_hist,train_acc_hist


#Plot the logs of the loss and accuracy on the train/validation set
def plot_hist(val_loss_hist,train_loss_hist,val_acc_hist,train_acc_hist):
    fig, (ax1, ax2) = plt.subplots(1, 2,figsize=(15,5))

    ax1.set_ylabel('Loss')
    ax2.set_ylabel('accuracy')

    ax1.plot(train_loss_hist,label='training')
    ax1.plot(val_loss_hist,label='validation')
    ax1.set_yscale('log')
    ax1.legend()

    ax2.plot(train_acc_hist,label='training')
    ax2.plot(val_acc_hist,label='validation')
    ax2.legend()

    plt.show()

Test

In [6]:
def test_and_save_predictions(network, test_imgs):

    process = psutil.Process(os.getpid())
    print(test_imgs.shape)
    softMax = torch.nn.Softmax(0)
    filenames_list = []
    for i in range(0, NR_TEST_IMAGES): # test_imgs.shape[0]
        print("Create prediction for image #", i+1)
        #filename = "/content/predictions/prediction_" + str(i+1) + ".png"
        filename = "../Datasets/Prediction/_" + str(i+1) + ".png"
        filenames_list.append(filename)
        print("Memory usage {0:.2f} GB".format(process.memory_info().rss/1024/1024/1024))

        image = test_imgs[i]
        image = torch.unsqueeze(image, 0)
        image = network(image)
        image = image[0]

        image = torch.argmax(softMax(image), 0)
        image = image.cpu().detach().numpy()
        image = image[2:610, 2:610]
        print(image.shape)
        Image.fromarray(255*image.astype('uint8')).save(filename)

    return filenames_list

def mean_std(x_ls):
    x_mean = torch.mean(torch.tensor(x_ls)).item()
    x_std = torch.std(torch.tensor(x_ls)).item()
    return x_mean, x_std


# Run the round times the whole process to see the variability with different initialisations (=> may not be feasible in our case)
def round_test(create_net, x, y, epoch, score, x_test, y_test):
    train_ls = []
    test_ls = []
    true_test_ls = []

    for i in range(ROUNDS):
        torch.manual_seed(2*i+1)
        network, loss_function, optimizer = create_net()

        print("Round {}".format(i),end="\r")
        val_loss_hist,train_loss_hist,val_acc_hist,train_acc_hist = training(network, loss_function, optimizer, score, x, y, epoch,val_split=0.01)

        #log
        train_ls.append(train_acc_hist[-1])
        test_ls.append(test(network,score,x_test,y_test))

        #Clear cache of GPU
        del network
        if torch.cuda.is_available():
            torch._C._cuda_emptyCache()

    return mean_std(train_ls),mean_std(test_ls),mean_std(true_test_ls)


Mask to submission

In [7]:
foreground_threshold = 0.25 # percentage of pixels > 1 required to assign a foreground label to a patch

# assign a label to a patch
def patch_to_label(patch):
    df = np.mean(patch)
    if df > foreground_threshold:
        return 1
    else:
        return 0


def mask_to_submission_strings(image_filename):
    """Reads a single image and outputs the strings that should go into the submission file"""
    img_number = int(re.search(r"\d+", image_filename).group(0))
    im = mpimg.imread(image_filename)
    patch_size = 16
    for j in range(0, im.shape[1], patch_size):
        for i in range(0, im.shape[0], patch_size):
            patch = im[i:i + patch_size, j:j + patch_size]
            label = patch_to_label(patch)
            yield("{:03d}_{}_{},{}".format(img_number, j, i, label))


def masks_to_submission(submission_filename, image_filenames):
    """Converts images into a submission file"""
    with open(submission_filename, 'w') as f:
        f.write('id,prediction\n')
        for fn in image_filenames:
            f.writelines('{}\n'.format(s) for s in mask_to_submission_strings(fn))


Submission to mask

In [8]:
h = 16
w = h
imgwidth = int(math.ceil((600.0/w))*w)
imgheight = int(math.ceil((600.0/h))*h)
nc = 3

# Convert an array of binary labels to a uint8
def binary_to_uint8(img):
    rimg = (img * 255).round().astype(np.uint8)
    return rimg

def reconstruct_from_labels(image_id, label_file):
    im = np.zeros((imgwidth, imgheight), dtype=np.uint8)
    f = open(label_file)
    lines = f.readlines()
    image_id_str = '%.3d_' % image_id
    for i in range(1, len(lines)):
        line = lines[i]
        if not image_id_str in line:
            continue

        tokens = line.split(',')
        id = tokens[0]
        prediction = int(tokens[1])
        tokens = id.split('_')
        i = int(tokens[1])
        j = int(tokens[2])

        je = min(j+w, imgwidth)
        ie = min(i+h, imgheight)
        if prediction == 0:
            adata = np.zeros((w,h))
        else:
            adata = np.ones((w,h))
        im[j:je, i:ie] = binary_to_uint8(adata)

    #Image.fromarray(im).save('/content/predictions/prediction_patches_' + '%.3d' % image_id + '.png')
    Image.fromarray(im).save('../Datasets/Prediction/prediction_patches_' + '%.3d' % image_id + '.png')

    return im


Run.py

In [9]:
def readTrainingImages(TRAINING_SIZE, data_dir, path, rotate = False, save = False):
    train_data_filename = data_dir + path
    to_tensor = transforms.ToTensor() #ToTensor transforms the image to a tensor with range [0,1]
    num_images = TRAINING_SIZE
    imgs = []
    r_imgs = []

    for i in range(1, num_images+1):

        imageid = "satImage_%.3d" % i
        image_filename = train_data_filename + imageid + ".png"
        print(image_filename)
        if os.path.isfile(image_filename):
            img = Image.open(image_filename)
            t_img = to_tensor(img).to(DEVICE) #3 [rgb] x 400 x 400
            
            imgs.append(t_img)

            if (rotate == True):
                width, height = img.size
                # Rotate images

                for degree in range(360):
                    rotated_img = img.rotate(degree)
                    # Crop to remove black parts
                    left     = width / 4 * 0.58
                    top      = height / 4 * 0.58
                    right    = width - left
                    bottom   = height - top
                    rotated_img = rotated_img.crop((left, top, right, bottom))
                    if (save == True):
                        # optional, save new image on disk, to see the effect
                        rotated_img.save("/content/Rotations/"+path + imageid + "_"+str(degree)+".png")
                    rt_img = to_tensor(rotated_img) # 3 [rgb] x 284 x 284
                    r_imgs.append(rt_img)
        else:
            print(image_filename, "is not a file, follow the README instruction to run the project. (check path)", file=sys.stderr)
            sys.exit()

    if rotate:
        r_imgs = torch.stack(r_imgs)
    imgs = torch.stack(imgs)

    return imgs, r_imgs


def readTestImages(test_directory, num_images):
    imgs = []
    to_tensor = transforms.ToTensor()
    current_image_path = ""
    for i in range(1, num_images+1):
        current_image_path = test_directory + str(i) + "/test_" + str(i) + ".png"
        if os.path.isfile(current_image_path):
            img = Image.open(current_image_path)
            t_img = to_tensor(img).to(DEVICE) # 3 [rgb] x 600 x 600
            imgs.append(t_img)

        else:
            print(current_image_path, "is not a file, follow the README instruction to run the project. (check path)", file=sys.stderr)
            sys.exit()

    imgs = torch.stack(imgs)
    return imgs

    # Assign a one-hot label to each pixel of a ground_truth image
    # can be improved using scatter probably
    # or see how it is done in the tf_aerial.py
def value_to_class(img):
    img = img.squeeze()
    H = img.shape[0]
    W = img.shape[1]
    labels = torch.randn((H,W)).to(DEVICE)
    foreground_threshold = 0.5
    for h in range(H) :
        for w in range(W) :
            if img[h,w] > foreground_threshold:  # road
                labels[h,w] = torch.tensor(1.0)
            else:  # bgrd
                labels[h,w] = torch.tensor(0.0)
    return labels.long()


def main():
    
    # process = psutil.Process(os.getpid()) ## in case we need to verify memory usage
    # print(process.memory_info().rss/1024/1024)  # in Mbytes

    # Reading test images
    test_imgs = readTestImages(test_dir, NR_TEST_IMAGES)

    # Reading training images
    train_imgs, r_imgs = readTrainingImages(TRAINING_SIZE, data_dir, train_data_filename, rotateFlag) # satellite
    print("Train device", train_imgs.device)
    labels, r_labels = readTrainingImages(TRAINING_SIZE, data_dir, train_labels_filename, rotateFlag) # labels
    print("Labels device", labels.device)
    # Preprocessing
    labels = F.pad(labels, (2, 2, 2, 2), mode = 'reflect') 
    print("Labels padded device", labels.device)
    labels_bin =  torch.stack([value_to_class(labels[i]) for i in range(TRAINING_SIZE)]) # decimal to binary
    labels_bin.to(DEVICE)
    print("labels bin device", labels_bin.device)


    # Creating the outline of the model we want
    model = create_UNET() # 5 layers
    loss = torch.nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters())
    # model, loss, optimizer = create_smallerUNET() # 4 layers
    summary(model, input_size=(3,400,400)) # prints memory resources
    
    # training all (TRAINING_SIZE) images
    val_loss_hist,train_loss_hist,val_acc_hist,train_acc_hist = training(model, loss, optimizer, train_imgs, labels_bin, NUM_EPOCHS, RATIO)
    
    # predicting on thw the test images
    filenames_list = test_and_save_predictions(model, test_imgs)

    # Create csv files
    masks_to_submission(submissionFileName, filenames_list)

    # create label images from csv
    for i in range(1, NR_TEST_IMAGES+1):
        reconstruct_from_labels(i, submissionFileName)

    
    return val_loss_hist,train_loss_hist,val_acc_hist,train_acc_hist


if __name__== "__main__":
        val_loss_hist,train_loss_hist,val_acc_hist,train_acc_hist = main()
        print("Training loss = ", train_loss_hist)
        print("Testing loss = ", val_loss_hist)        
        print("Training accuracy = ", train_acc_hist)
        print("Testing accuracy = ", val_acc_hist)



../Datasets/training/images/satImage_001.png
../Datasets/training/images/satImage_002.png
../Datasets/training/images/satImage_003.png
../Datasets/training/images/satImage_004.png
Train device cpu
../Datasets/training/groundtruth/satImage_001.png
../Datasets/training/groundtruth/satImage_002.png
../Datasets/training/groundtruth/satImage_003.png
../Datasets/training/groundtruth/satImage_004.png
Labels device cpu
Labels padded device cpu
labels bin device cpu
layer0 torch.Size([2, 3, 588, 588])
layer1d torch.Size([2, 64, 584, 584])
layer2d torch.Size([2, 128, 288, 288])
layer3d torch.Size([2, 256, 140, 140])
layer4d torch.Size([2, 512, 66, 66])
layer5 torch.Size([2, 512, 33, 33])
layer4a torch.Size([2, 512, 58, 58])
layer3a torch.Size([2, 256, 108, 108])
layer2a torch.Size([2, 128, 208, 208])
layer1a torch.Size([2, 64, 408, 408])
output torch.Size([2, 2, 404, 404])
----------------------------------------------------------------
        Layer (type)               Output Shape         Par

layer2a torch.Size([1, 128, 208, 208])
layer1a torch.Size([1, 64, 408, 408])
output torch.Size([1, 2, 404, 404])
layer0 torch.Size([1, 3, 588, 588])
layer1d torch.Size([1, 64, 584, 584])
layer2d torch.Size([1, 128, 288, 288])
layer3d torch.Size([1, 256, 140, 140])
layer4d torch.Size([1, 512, 66, 66])
layer5 torch.Size([1, 512, 33, 33])
layer4a torch.Size([1, 512, 58, 58])
layer3a torch.Size([1, 256, 108, 108])
layer2a torch.Size([1, 128, 208, 208])
layer1a torch.Size([1, 64, 408, 408])
output torch.Size([1, 2, 404, 404])
layer0 torch.Size([1, 3, 588, 588])
layer1d torch.Size([1, 64, 584, 584])
layer2d torch.Size([1, 128, 288, 288])
layer3d torch.Size([1, 256, 140, 140])
layer4d torch.Size([1, 512, 66, 66])
layer5 torch.Size([1, 512, 33, 33])
layer4a torch.Size([1, 512, 58, 58])
layer3a torch.Size([1, 256, 108, 108])
layer2a torch.Size([1, 128, 208, 208])
layer1a torch.Size([1, 64, 408, 408])
output torch.Size([1, 2, 404, 404])
>> Saving Model for Epoch  1
Training, epoch= 2
Memory usag